In [ ]:
import os, random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers

(train_X, train_y), (test_X, test_y) = keras.datasets.cifar10.load_data()

train_y = train_y.squeeze().astype(np.int64)
test_y = test_y.squeeze().astype(np.int64)
# normalization
train_X = train_X.astype(np.float32) / 255.0
test_X = test_X.astype(np.float32) / 255.0

class_names = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

print("Train:", train_X.shape, train_y.shape)
print("Test :", test_X.shape, test_y.shape)
print("Classes:", class_names[:5], "...")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Train: (50000, 32, 32, 3) (50000,)
Test : (10000, 32, 32, 3) (10000,)
Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer'] ...


In [3]:
def set_seed(seed=42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)

NUM_CLASSES = 10
INPUT_SHAPE = (32, 32, 3)

def build_model(use_dropout=False, use_batchnorm=False, dropout_rate=0.3):
    def maybe_bn(x):
        return layers.BatchNormalization()(x) if use_batchnorm else x

    inputs = keras.Input(shape=INPUT_SHAPE)
    x = inputs

    x = layers.Conv2D(32, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(32, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D((2,2))(x)
    if use_dropout:
        x = layers.Dropout(dropout_rate)(x)

    x = layers.Conv2D(64, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(64, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D((2,2))(x)
    if use_dropout:
        x = layers.Dropout(dropout_rate)(x)

    x = layers.Conv2D(128, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.Conv2D(128, (3,3), padding="same", use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)

    x = layers.MaxPooling2D((2,2))(x)
    if use_dropout:
        x = layers.Dropout(dropout_rate)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(128, use_bias=not use_batchnorm)(x)
    x = maybe_bn(x)
    x = layers.Activation("relu")(x)
    if use_dropout:
        x = layers.Dropout(dropout_rate)(x)

    outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model

def compile_model(model, lr=1e-3):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

def train_and_eval(model, name, train_X, train_y, test_X, test_y, epochs=20, batch_size=64):
    compile_model(model)

    callbacks = [
        keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
    ]

    print(f"\n=== {name} ===")
    print("Params:", model.count_params())
    history = model.fit(
        train_X, train_y,
        validation_split=0.1,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )

    test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
    print(f"{name} | Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

    best_val_acc = max(history.history["val_accuracy"])
    best_val_loss = min(history.history["val_loss"])
    return {
        "name": name,
        "params": model.count_params(),
        "best_val_acc": float(best_val_acc),
        "best_val_loss": float(best_val_loss),
        "test_acc": float(test_acc),
        "test_loss": float(test_loss),
    }

models = [
    ("Baseline", build_model(use_dropout=False, use_batchnorm=False)),
    ("+ Dropout(0.3)", build_model(use_dropout=True, use_batchnorm=False, dropout_rate=0.3)),
    ("+ BatchNorm", build_model(use_dropout=False, use_batchnorm=True)),
    ("+ BatchNorm + Dropout(0.3)", build_model(use_dropout=True, use_batchnorm=True, dropout_rate=0.3)),
]

results = []
for name, m in models:
    results.append(train_and_eval(m, name, train_X, train_y, test_X, test_y, epochs=30, batch_size=64))

print("\n=== PODSUMOWANIE ===")
for r in results:
    print(
        f"{r['name']:<28} params={r['params']:<9} "
        f"best_val_acc={r['best_val_acc']:.4f} test_acc={r['test_acc']:.4f}"
    )


=== Baseline ===
Params: 550570
Epoch 1/30
704/704 - 211s - 300ms/step - accuracy: 0.4314 - loss: 1.5432 - val_accuracy: 0.4986 - val_loss: 1.3286 - learning_rate: 1.0000e-03
Epoch 2/30
704/704 - 201s - 285ms/step - accuracy: 0.6214 - loss: 1.0601 - val_accuracy: 0.6600 - val_loss: 0.9640 - learning_rate: 1.0000e-03
Epoch 3/30
704/704 - 202s - 287ms/step - accuracy: 0.7015 - loss: 0.8480 - val_accuracy: 0.7210 - val_loss: 0.8092 - learning_rate: 1.0000e-03
Epoch 4/30
704/704 - 206s - 292ms/step - accuracy: 0.7517 - loss: 0.7062 - val_accuracy: 0.7496 - val_loss: 0.7490 - learning_rate: 1.0000e-03
Epoch 5/30
704/704 - 198s - 281ms/step - accuracy: 0.7916 - loss: 0.5930 - val_accuracy: 0.7510 - val_loss: 0.7593 - learning_rate: 1.0000e-03
Epoch 6/30

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
704/704 - 207s - 294ms/step - accuracy: 0.8171 - loss: 0.5160 - val_accuracy: 0.7352 - val_loss: 0.8293 - learning_rate: 1.0000e-03
Epoch 7/30
704/704 - 200s - 284m

In [ ]:
def build_model(use_dropout=False, use_batchnorm=False, dropout_rate=0.3):
    inputs = keras.Input(shape=(32,32,3))
    x = inputs

    
    x = layers.Conv2D(16,(3,3),padding="same",activation="relu")(x)
    x = layers.Conv2D(16,(3,3),padding="same",activation="relu")(x)
    x = layers.MaxPooling2D((2,2))(x)
    if use_dropout: x = layers.Dropout(dropout_rate)(x)

    
    x = layers.Conv2D(32,(3,3),padding="same",activation="relu")(x)
    x = layers.Conv2D(32,(3,3),padding="same",activation="relu")(x)
    x = layers.MaxPooling2D((2,2))(x)
    if use_dropout: x = layers.Dropout(dropout_rate)(x)

    
    x = layers.Conv2D(64,(3,3),padding="same",activation="relu")(x)
    x = layers.MaxPooling2D((2,2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    if use_dropout: x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    return keras.Model(inputs, outputs)

def compile_model(model, lr=1e-3):
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

def train_and_eval(model, name, train_X, train_y, test_X, test_y, epochs=20, batch_size=64):
    compile_model(model)

    callbacks = [
        keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1)
    ]

    print(f"\n=== {name} ===")
    print("Params:", model.count_params())
    history = model.fit(
        train_X, train_y,
        validation_split=0.1,
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )

    test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
    print(f"{name} | Test accuracy: {test_acc:.4f} | Test loss: {test_loss:.4f}")

    # The best version
    best_val_acc = max(history.history["val_accuracy"])
    best_val_loss = min(history.history["val_loss"])
    return {
        "name": name,
        "params": model.count_params(),
        "best_val_acc": float(best_val_acc),
        "best_val_loss": float(best_val_loss),
        "test_acc": float(test_acc),
        "test_loss": float(test_loss),
    }

# different models
models = [
    ("Baseline", build_model(use_dropout=False, use_batchnorm=False)),
    ("+ Dropout(0.3)", build_model(use_dropout=True, use_batchnorm=False, dropout_rate=0.3)),
    ("+ BatchNorm", build_model(use_dropout=False, use_batchnorm=True)),
    ("+ BatchNorm + Dropout(0.3)", build_model(use_dropout=True, use_batchnorm=True, dropout_rate=0.3)),
]

results = []
for name, m in models:
    results.append(train_and_eval(m, name, train_X, train_y, test_X, test_y, epochs=15, batch_size=32))

print("\n=== Summary ===")
for r in results:
    print(
        f"{r['name']:<28} params={r['params']:<9} "
        f"best_val_acc={r['best_val_acc']:.4f} test_acc={r['test_acc']:.4f}"
    )


=== Baseline ===
Params: 101402
Epoch 1/15
1407/1407 - 111s - 79ms/step - accuracy: 0.4466 - loss: 1.5232 - val_accuracy: 0.5808 - val_loss: 1.1768 - learning_rate: 1.0000e-03
Epoch 2/15
1407/1407 - 107s - 76ms/step - accuracy: 0.6200 - loss: 1.0764 - val_accuracy: 0.6502 - val_loss: 0.9826 - learning_rate: 1.0000e-03
Epoch 3/15
1407/1407 - 107s - 76ms/step - accuracy: 0.6798 - loss: 0.9118 - val_accuracy: 0.6884 - val_loss: 0.8949 - learning_rate: 1.0000e-03
Epoch 4/15
1407/1407 - 109s - 77ms/step - accuracy: 0.7163 - loss: 0.8108 - val_accuracy: 0.7148 - val_loss: 0.8121 - learning_rate: 1.0000e-03
Epoch 5/15
1407/1407 - 109s - 78ms/step - accuracy: 0.7409 - loss: 0.7370 - val_accuracy: 0.7252 - val_loss: 0.7953 - learning_rate: 1.0000e-03
Epoch 6/15
1407/1407 - 107s - 76ms/step - accuracy: 0.7658 - loss: 0.6705 - val_accuracy: 0.7260 - val_loss: 0.8130 - learning_rate: 1.0000e-03
Epoch 7/15
1407/1407 - 143s - 102ms/step - accuracy: 0.7805 - loss: 0.6214 - val_accuracy: 0.7546 - val